# Chapter 5

### 名目類型特徵編碼(LabelBinarizer)

In [16]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
feature = np.array([["Texas"], ["California"], ["Texas"], ["Delaware"], ["Texas"]])
one_hot = LabelBinarizer()

In [18]:
# 用one_hot 做特徵編碼
one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [20]:
# 檢視特徵類型
one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

In [22]:
# 反向編碼
one_hot.inverse_transform(one_hot.fit_transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

### 甚至可以使用Pandas做

In [24]:
feature

array([['Texas'],
       ['California'],
       ['Texas'],
       ['Delaware'],
       ['Texas']], dtype='<U10')

In [30]:
import pandas as pd
pd.get_dummies(feature[:, 0]) # 出來直接是一個DataFrame

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


### 多重特徵編碼

In [32]:
from sklearn.preprocessing import MultiLabelBinarizer
muticlass_feature = [("Texas", "Florida"),
                    ("California", "Alabama"),
                    ("Texas", "Florida"),
                    ("Delware", "Florida"),
                    ("Texas", "Alabama")]
one_hot_multicalass = MultiLabelBinarizer()
one_hot_multicalass.fit_transform(muticlass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [33]:
one_hot_multicalass.classes_

array(['Alabama', 'California', 'Delware', 'Florida', 'Texas'],
      dtype=object)

###  有序類型特徵編碼

In [37]:
df = pd.DataFrame({"Score":["Low", "Low", "Medium", "Medium", "High"]})
scale_mapper = {"Low": 1, "Medium":2, "High":3 }
df["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

### 特徵字典編碼 

In [49]:
from sklearn.feature_extraction import DictVectorizer

data_dict = [{"Red":2, "Blue":4}, {"Red":4, "Blue":3}, {"Red":1, "Yellow":2}, {"Red":2, "Yellow":2}]
dictvectorizer = DictVectorizer(sparse=False) # Dense matrix 降低記憶體需求
features = dictvectorizer.fit_transform(data_dict)
features

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

In [51]:
# 取得特徵名稱
feature_names = dictvectorizer.get_feature_names()
feature_names

['Blue', 'Red', 'Yellow']

In [52]:
pd.DataFrame(features, columns=feature_names)

,Blue,Red,Yellow
0,4.0,2.0,0.0
1,3.0,4.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


In [53]:
# 假設有四份文件(NLP)
doc_1 = {"Red":2, "Blue":4}
doc_2 = {"Red":4, "Blue":3}
doc_3 = {"Red":1, "Yellow":2}
doc_4 = {"Red":2, "Yellow":2}
doc_word_counts = [doc_1, doc_2, doc_3, doc_4]
dictvectorizer.fit_transform(doc_word_counts)

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

### 缺漏類型值的推算 

In [57]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1,-0.21,-1.19]])
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                       [np.nan,-0.67,-0.22]])
clf = KNeighborsClassifier(3, weights ='distance')
trained_model = clf.fit(X[:, 1:], X[:, 0])
# 預測模型
imputed_values = trained_model.predict(X_with_nan[:, 1:])
# 合成預測值跟原始
X_with_imputed = np.hstack((imputed_values.reshape(-1, 1), X_with_nan[:, 1:]))
# 兩個矩陣[X_with_nan, X]結合
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

#### 用特徵頻率最高的值來補空值 

In [60]:
from sklearn.preprocessing import Imputer
X_complete = np.vstack((X_with_nan, X)) # 含空值的合併
imputer = Imputer(strategy='most_frequent', axis=0)
imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

#### Notes:最好使用機器學習演算法來預測及填補資料，最常用KNN

### 處理高度不平衡類型的目標向量 

In [64]:
# 生成資料
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

In [79]:
iris = load_iris()
features = iris.data
target = iris.target

In [80]:
features = features[40:, :]
target = target[40:]
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [81]:
target = np.where((target==0), 0, 1)
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [86]:
# 產生權重(?)
weights={0: .9, 1: 0.1}
RandomForestClassifier(class_weight=weights)
# 自動產生死類型出現頻率成反比的權重
# RandomForestClassifier(class_weight='balanced')

RandomForestClassifier(bootstrap=True, class_weight={0: 0.9, 1: 0.1},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators='warn', n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

#### Notes:或者我們可以向下取樣，針對多數類型進行隨機取樣已產生相當於少數類型的新子集 

#### 向下取樣

In [103]:
# 跟下面比較看看哪裡不同
i_class0 = np.where(target==0)
i_class0

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),)

In [104]:
i_class0 = np.where(target==0)[0]
i_class1 = np.where(target==1)[0]
i_class0

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [105]:
print(len(i_class0), len(i_class1))

10 100


In [106]:
i_class1_downsampled = np.random.choice(i_class1, size=len(i_class0), replace=False) # replace=False取後不放回

In [107]:
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [109]:
np.vstack((features[i_class0, :], features[i_class1_downsampled, :]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

#### 向上取樣

In [113]:
i_class0_upsampled = np.random.choice(i_class0, size=len(i_class1), replace=True) # replace=True取後放回
np.hstack((target[i_class0_upsampled], target[i_class1]))
# 也可以用concatenate!
# np.concatenate((target[i_class0_upsampled], target[i_class1]))
np.vstack((features[i_class0_upsampled, :], features[i_class1, :]))[0:5]

array([[5. , 3.5, 1.6, 0.6],
       [5. , 3.5, 1.6, 0.6],
       [5. , 3.5, 1.6, 0.6],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6]])

### 處理不平衡的策略 

1. 增加數據
2. 使用模型評估指標(Confusion Matrix, F1-score, ROC curves)
3. 使用類型權重參數，(class_weight=?)
4. 向上或是向下取樣(通常兩種都會試)